In [1]:
import pandas as pd
import numpy  as np
import re
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"

In [2]:
file_path = 'data/Restaurant_Reviews.csv'
yorumlar  = pd.read_csv(file_path, on_bad_lines='skip')

In [3]:
df = yorumlar.copy()

In [10]:
df['Review'].shape

(718,)

In [33]:
def get_list_sentence(sentence):
        return re.sub('[^a-zA-Z]', " ", sentence).lower().split()

def get_new_df(column, index=True, index_value=0):
    if index:
        return get_list_sentence(column[index_value])
    else:
        liste = []
        for row in column:
            liste.append(get_list_sentence(row))
        return liste
    
# get_new_df(df['Review'], index=False)
yorum = get_new_df(df['Review'])
yorum

['wow', 'loved', 'this', 'place']

In [8]:
import nltk
durma = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kerim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [111]:
colon = df[[' Liked']]
def col_round(x):
    frac = x - math.floor(x)
    if frac < 0.5: 
        return math.floor(x)
    return math.ceil(x)

df[[' Liked']].isna().sum()
colon_ort = colon.mean().item()
df.fillna(col_round(colon_ort), inplace=True)
df[[' Liked']].isna().sum()

 Liked    12
dtype: int64

 Liked    0
dtype: int64

In [112]:
from nltk.stem.porter import PorterStemmer


from nltk.corpus      import stopwords
ps = PorterStemmer()
yorum
yorum = [ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('english'))]
yorum

def get_modified_sentence(sentence):
    return [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words('english'))]


['wow', 'love', 'place']

['wow', 'love', 'place']

In [113]:
get_modified_sentence(yorum)
" ".join(yorum)

['wow', 'love', 'place']

'wow love place'

## Tum satırları için

In [114]:
modified_column_values = get_new_df(df['Review'], index=False)
result_list = []
for row in modified_column_values:
    row = get_modified_sentence(row)
    result_list.append(" ".join(row))

In [115]:
# result_list

### Feature Extraction ( Öznitelik Çıkarımı )
### Bag Of Word ( BOW )

In [116]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(result_list).toarray() # Bagımsız değişken
y = df.iloc[:,1].values # Bagımlı değişken

In [117]:
from sklearn.model_selection import train_test_split

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [119]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

GaussianNB()

In [123]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[42, 23],
       [16, 63]])